In [1]:
import numpy as np
import sys

try:
    import mosek.fusion as mf

    expr = mf.Expr
    dom = mf.Domain
    mat = mf.Matrix
except Exception as e:
    import logging

    logging.exception(e)

In [87]:
def subp_x(
        xi,
        kappa,
        mu,
        rho=1,
        verbose=True
):
  n, d = xi.shape
  model = mf.Model("kissing-num")
  S = model.variable('s', dom.greaterThan(0))
  X = model.variable('x', [n, d])

  # |xi + xj | <= 12
  for i in range(n):
    for j in range(i, n):
      model.constraint(
        expr.vstack(
          0.5,
          S,
          expr.flatten(expr.add(X.slice([i, 0], [i + 1, d]), X.slice([j, 0], [j + 1, d])))
        ),
        dom.inRotatedQCone()
      )
  model.constraint(S, dom.lessThan(12))


  obj_expr = 0

  # ALM terms
  # the t - s gap
  t = model.variable("t", dom.inRange(0, 4))
  expr_norm_gap = expr.sub(t, 4)
  expr_norm_gap_sqr = model.variable("t_s", dom.greaterThan(0))
  model.constraint(
    expr.vstack(1 / 2, expr_norm_gap_sqr, expr_norm_gap), dom.inRotatedQCone()
  )
  for i in range(n):
    model.constraint(
      expr.vstack(1/2, t, expr.flatten(X.slice([i, 0], [i + 1, d]))),
      dom.inRotatedQCone()
    )
  obj_expr = expr.add(obj_expr, expr.mul(kappa, expr_norm_gap))
  obj_expr = expr.add(obj_expr, expr.mul(rho / 2, expr_norm_gap_sqr))
  # the <𝜉, x> - gap
  expr_norm_x_gap_sqr = model.variable("xi_x", [n], dom.greaterThan(0))
  for i in range(n):
    expr_norm_x_gap = expr.sub(
      expr.dot(
        expr.flatten(X.slice([i, 0], [i + 1, d])),
        xi[i]
      ),
      4
    )
    model.constraint(
      expr.vstack(1 / 2, expr_norm_x_gap_sqr.index(i), expr_norm_x_gap),
      dom.inRotatedQCone()
    )
    obj_expr = expr.add(obj_expr, expr.mul(mu[i], expr_norm_x_gap))
    obj_expr = expr.add(obj_expr, expr.mul(rho / 2, expr_norm_x_gap_sqr.index(i)))

  # ALM objective
  model.objective(mf.ObjectiveSense.Minimize, obj_expr)
  if verbose:
    model.setLogHandler(sys.stdout)
  model.solve()
  x = X.level().reshape((n, d))
  x_s = expr_norm_x_gap_sqr.level()
  s = S.level()
  t = t.level()
  return x, x_s, t, s

In [99]:
def subp_xi(x, mu, t, rho=1, verbose=True):
  n, d = x.shape
  ###################
  # The above part is unchanged
  ###################
  # norm bounds on y^Te
  model = mf.Model("kissing-num-aux")
  Xi = model.variable("xi", [n, d])

  for i in range(n):
    model.constraint(
      expr.vstack(2, expr.flatten(Xi.slice([i, 0], [i+1, d]))),
      dom.inQCone()
    )
  obj_expr = 0

  # ALM terms
  # the <𝜉, x> - gap
  expr_norm_x_gap_sqr = model.variable("xi_x", [n], dom.greaterThan(0))

  for i in range(n):
    expr_norm_x_gap = expr.sub(
      expr.dot(
        x[i],
        expr.flatten(Xi.slice([i, 0], [i + 1, d]))
      ),
      4
    )
    model.constraint(
      expr.vstack(1 / 2, expr_norm_x_gap_sqr.index(i), expr_norm_x_gap),
      dom.inRotatedQCone()
    )
    obj_expr = expr.add(obj_expr, expr.mul(mu[i], expr_norm_x_gap))
    obj_expr = expr.add(obj_expr, expr.mul(rho / 2, expr_norm_x_gap_sqr.index(i)))

  # ALM objective
  model.objective(mf.ObjectiveSense.Minimize, obj_expr)
  if verbose:
    model.setLogHandler(sys.stdout)
  model.solve()
  xi = Xi.level().reshape((n, d))
  xs_sqr = expr_norm_x_gap_sqr.level()
  return xi, xs_sqr

In [75]:
n, d = 2, 2

In [98]:
xi = x = np.ones((n, d))
rho = 1
mu = np.ones(n)
kappa = 0
x, x_s, t, s = subp_x(x, kappa, mu, rho, True)

Problem
  Name                   : kissing-num     
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 30              
  Cones                  : 8               
  Scalar variables       : 39              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 3
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   : kissing-num     
  Objective se

In [97]:
x_s

array([2.40408227, 2.40408227])

In [96]:
((x@xi.T ) -4) ** 2


array([[2.40408234, 2.40408234],
       [2.40408234, 2.40408234]])

In [90]:
kappa * (t - 4) + rho/2*(t-4)**2 + np.diag(x @ xi.T).dot(mu) + sum(np.diag(x @ xi.T)**2 - 4) * rho

array([8.89897846])

In [78]:
for i in range(10):
  x, t, s = subp_x(xi, kappa, mu, rho, False)
  xi, xs_sqr = subp_xi(x, mu, rho, False)
  x_s  = np.diag(x @ xi.T) - 4
  # t_s  = t - 4
  alm = x_s.T @ mu + xs_sqr.sum() * rho # + t_s @ kappa
  mu += x_s * rho
  rho = rho * 1.1
  # if abs(alm) < 1e-2:
  #   break
  print(f"@{i}, alm: {alm}, bd: {0}, 𝜉x: {x_s + 4}")

DimensionError: Mismatching operand shapes

In [64]:
x_s

array([-0.53589839, -0.53589839])

In [18]:
np.sqrt(3/2)

1.224744871391589